# Simulating Planetary Orbits

Author: Ethan Huang Kao

Date: 13th of December, 2022

Completed as part of PHAS0007 Computing Final Assignment

## Introduction

According to Isaac Newton, any two masses experience a force of attraction between them. The force experienced by these objects is given by Newton's Law of Gravitation [1]:

$$\vec{F} = G \frac{m_1 \times m_2}{\vec{r^2}}$$
   
where G is Newton's gravitational constant which is approximately $6.6743 \times 10^{-11} N m^2 kg^{-2}$. This equation can also be expressed in the following vector form [1]:
$$ \vec{F}_{12} = -G \frac{m_1 \times m_2}{|\vec{r}_{21}|^2} \hat{r}_{21}(1)$$
 
Thus, the acceleration of object $m_1$ small in mass caused by $m_2$ can be found using the following equations:

$$ \vec{F} = m_1 \times \vec{a_1} (2)$$
$$ -G \frac{m_1 \times m_2}{|\vec{r}_{21}|^2} \hat{r}_{21} = m_1 \times \vec{a_1} (3)$$
$$ -G \frac{m_2}{|\vec{r}_{21}|^2} \hat{r}_{21} = \vec{a_1} (4)$$

Equation (1) is subbed into equation (2) to give equation (3). We can see that the masses of $m_1$ cancel out to give equation (4). This means that the acceleration of the small object is only dependent on the mass of the large object ($m_2$) and the distance between the two objects.

We will now consider Euler's method of a decreasing step size. If we assume that the acceleration of $m_2$ on $m_1$ is constant for a small time step $dt$, we can use projectile motion equations to calculate the objects changing velocity and changing position. The following equations can be used:

$$ \vec{v} = \vec{u} + \vec{a_1} \cdot dt$$
$$ \vec{r} = \vec{r_o} + \vec{u}\cdot dt$$

I will first investigate the force experienced by one planet by one star. This investigation will be followed by illustrating how two planets would interact with each other in the prescence of a star's gravitaitonal field. We will then investigate how best to animate a system of a star, planet, and a moon. Finally, I will touch on the three-body problem and investigate how changing initial parameters can drastically change the outcome.

In [ ]:
#initialise packages, provided by [2]

#packages for animation
%matplotlib notebook
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

#packages for math functions
import numpy as np
from numpy.linalg import norm

## Part A: Orbiting planet around a star

### A1: Calculating the gravitational force between two objects
Written below is a function which calculates the gravitaitonal force between two masses at two different positions. Gravitational force is found using Newton's Law of Gravitaiton [1].

In [ ]:
def force(pos1, pos2, m1, m2):
    """
    Returns the gravitational force exerted by object 2 on object 1.
    Input:
      - pos1: position vector of first object  (NumPy array) [m]
      - pos2: position vector of second object (NumPy array) [m]
      - m1:   mass of first object (float) [kg]
      - m2:   mass of second object (float) [kg]
    Depends on:
      - G:    gravitational constant (global variable)
    """
    #get vector 'r' between two masses
    pos_diff = np.array(pos2)-np.array(pos1)
    
    #calculate modulus of 'r'
    r_mod = np.linalg.norm(pos_diff)
    
    #calculate unit vector of 'r'
    r_unit = (1/r_mod)*pos_diff
    
    #apply newton's law of gravitation to find force of gravity
    g_force = (r_unit*G * m1 * m2) / r_mod**2
    return g_force

#### A1: Functional tests
The following cell applies some tests to ensure that the function written above works properly. In general, the tests ensure that changing position and mass gives the expected result. In addition, the gravitational constant (G) has been set to 1 to make it more obvious if the function works properly. Tests provided by [2].

In [ ]:
def test_force(pos1, pos2, m1, m2, expected_force):
    """Check whether force function gives expected results."""
    epsilon = 1e-10
    f = force(np.array(pos1), np.array(pos2), m1, m2)
    if not isinstance(f,np.ndarray):
        print(f"ERROR: function should return a vector but returns {f}.")
        return
    args_as_string = f"({pos1}, {pos2}, {m1}, {m2})"
    error = norm(f-expected_force)
    if error<epsilon:
        print(f"OK: correct results for input {args_as_string}")
    else:
        print(f"ERROR: wrong results for input {args_as_string}")
        print(f"  expected: {expected_force}")
        print(f"  got:      {f}")

# Set global variable G to 1 for now to make testing easier
G=1

# Test force with some simple cases
test_force((0,0),(1,0),1,1,(1,0))    # distance = 1 in x direction
test_force((1,0),(0,0),1,1,(-1,0))   # swap objects
test_force((0,0),(2,0),1,1,(0.25,0)) # distance = 2
test_force((0,0),(0,1),1,1,(0,1))    # distance = 1 in y direction
test_force((10,0),(10,1),1,1,(0,1))  # displaced from origin
test_force((0,0),(1,0),2,1,(2,0))    # non-unit mass 1
test_force((0,0),(1,0),1,2,(2,0))    # non-unit mass 2

## A2: Calculating the motion of a planet in the gravitational field of a star

The following equation utilises the function `force` to calculate the motion of a planet. This is done by assuming a constant acceleration across a small time step `dt` and thus calculating the new velocity and position. This process of decreasing a small time step `dt` is known as Euler's method.

In addition, it is assumed that the star is positioned at the origin and the planet orbits around it.

In [ ]:
def move_planet(position, velocity, m_star, dt):
    """
    Calculate motion of planet in the gravitational field of a star with given mass
    at the origin, using Euler's method.
    
    Input:
      - position: position vector of planet at start of time step (NumPy array)
      - velocity: velocity vector of planet at start of time step (NumPy array)
      - m_star:   mass of star
      - dt:       time step
      
    Output: (position_new, velocity_new)
      - position_new: position vector of planet at end of time step (NumPy array)
      - velocity_new: velocity vector of planet at end of time step (NumPy array)
      
    Depends on:
      - force: function to calculate the gravitational force between two objects based
      on their positions and masses
    """
    #set starting star posiiton
    star_pos = np.array([0,0])
    
    #calculate acceleration
    #acceleration = gravitational force when orbiting object has mass of 1
    star_accel = force(position, star_pos,1, m_star)
    
    #calculate new velocity
    velocity_new = velocity + star_accel*dt
    #calculate new position
    position_new = np.array(position) + np.array(velocity)*dt
    return position_new, velocity_new

#### A2: Functional tests

The following cell applies some tests to ensure the function above works correctly. In general, the tests ensure that changing position and mass gives the expected result. In addition, the gravitational constant (G) has been set to 1 to make it more obvious if the function works properly. Tests provided by [2].

In [ ]:
def test_move_planet(position, velocity, m_star, dt, expected_pos, expected_vel):
    """Check whether move_planet function gives expected results."""
    epsilon = 1e-10
    results = move_planet(position, velocity, m_star, dt)
    if not isinstance(results, tuple):
        print(f"ERROR: function should return two vectors but returns {results}.")
        return
    if not len(results)==2:
        print(f"ERROR: function should return two vectors but returns {results}.")
        return
    pos, vel = results
    if not (isinstance(pos,np.ndarray) and isinstance(vel,np.ndarray)):
        print(f"ERROR: function should return two vectors but returns {results}.")
        return
    args_as_string = f"{position}, {velocity}, {m_star}, {dt}"
    err_pos, err_vel = norm(pos - expected_pos), norm(vel - expected_vel)
    if err_pos < epsilon and err_vel < epsilon:
        print(f"OK: correct results for input {args_as_string}")
    else:
        print(f"ERROR: wrong results for input {args_as_string}")
        print(f"  expected: {expected_pos, expected_vel}")
        print(f"  got:      {results}")

# Set global variable G to 1 for now to make testing easier
G=1

# Test move_planet with some simple cases
test_move_planet((1,0), (1,0), 1, 0, (1,0), (1,0))    # dt = 0: output = input
test_move_planet((1,0), (1,0), 1, 1, (2,0), (0,0))    # moving away from star
test_move_planet((0,1), (0,-1), 1, 1, (0,0), (0,-2))  # moving towards star
test_move_planet((1,0), (0,1), 1, 1, (1,1), (-1,1))   # moving past star
test_move_planet((1,0), (0,1), 1, 0.1, (1,0.1), (-0.1,1))  # smaller dt
test_move_planet((1,0), (0,1), 2, 0.1, (1,0.1), (-0.2,1))  # larger star mass
test_move_planet((2,0), (1,0), 1, 1, (3,0), (0.75,0)) # non-unit distance

### A3: Calculating the orbit of a planet

The following code defines a new function which generates a list of x-coordinates and y-coordinates of the orbiting planet using the function `move_planet`.

In [ ]:
def trajectory(position, velocity, m_star, dt, t_max):
    """
    Calculate trajectory of planet from given starting position, using Euler's method.
    
    Input:
      - position: position vector of planet at start of simulation [m] (NumPy array)
      - velocity: velocity vector of planet at start of simulation [m] (NumPy array)
      - m_star:   mass of star (float) [kg]
      - dt:       time step    (float) [s]
      - t_max:    duration of calculated motion (float) [s]

    Output: (x_arr, y_arr) [m]
        where x_arr and y_arr are NumPy arrays containing the x and y coordinates of
        the planet at each time step, starting with the initial position
    
    Depends:
      - move_planet: function which calculates the new position and velocity of a planet
      of given initial position and velocity after time dt
    """
    #array of positions
    x_arr, y_arr = [position[0]] , [position[1]]
    #total runtime in s
    time = 0
    while time <= t_max:
        #call function to calculate new position and velocity
        position, velocity = move_planet(position, velocity, m_star, dt)
        x,y = position
        x_arr.append(x)
        y_arr.append(y)       
        time += dt
    return x_arr, y_arr

#### A3: Functional test
I will now use the function `trajectory` to ensure that the function creates an almost circular orbit. Test provided by [2].

In [ ]:
G       = 6.6743e-11 # gravitational constant [m^3 kg^-1 s^-2]
YEAR    = 3.154e7    # one year in seconds [s]
M_STAR  = 2.5e30     # mass of star [kg]

dt_e = YEAR/10000      # time step for Euler's method [s]
pos = (2.0e11, 0)    # initial position (x,y) [m]
vel = (0,2.89e4)     # initial velocity (vx,vy) [m/s]

# Calculate trajectory
x_test, y_test = trajectory(pos,vel,M_STAR,dt_e,2*YEAR)

# Plot trajectory
plt.ion()
fig, axes = plt.subplots()
axes.set_aspect('equal')
plt.title('Trajectory of planet')
plt.xlabel('x [m]')
plt.ylabel('y [m]')
plt.plot(x_test, y_test, label="Trajectory of planet")

#add circle at center to show position of star
circle1 = plt.Circle((0, 0), 5e9, color='r', label="Star (not to scale)")
axes.add_patch(circle1)

plt.legend(loc="upper right")

### A4: Animating the orbit
To better visualise the motion of the planet, we will use `matplotlib` to illustrate the planet's movement. Two functions are defined below: a function to create a trail and a function to move the planet.

In [ ]:
#code taken from [2]
def animate_with_trail(i, planet, trail, x_arr, y_arr):
    """
    Update display for projectile motion, with trail.
    
    Arguments:
        i:      frame number (from 0 at time = 0)
        planet: Line2D object containing coordinates of planet
        trail:  Line2D object containing coordinates of trail
        x_arr:  array of x coordinate at teach time step
        y_arr:  array of y coordinate at teach time step

    Result: updates coordinates in Line2D objects provided as arguments
    "bodies" and "trails".
    """
    x, y = x_arr[i], y_arr[i]
    planet.set_data([x],[y])                # Body gets coordinates of current position
    trail.set_data(x_arr[:i+1],y_arr[:i+1]) # Trail has all points up to the current one

def create_animation(x_arr, y_arr, dt, fps, speedup):
    """
    Create an animation of an object with given trajectory.
    
    Arguments:
        x_arr, y_arr: arrays of x and y coordinates at each time step [m]
        dt:           time step in simulation [s]
        fps:          number of frames per second for animation
        speedup:      ratio of simulated time to screen time
    """
    # Find range of coordinates to include full trajectory
    x_max = max(np.abs(x_arr))
    y_max = max(np.abs(y_arr))
    r_max = 1.05 * max(x_max, y_max)
    
    # Create and configure figure and axes
    plt.ioff()
    fig, axes = plt.subplots()
    axes.set_xlim(-r_max,r_max)
    axes.set_ylim(-r_max,r_max)
    axes.set_aspect('equal')
    axes.set_title('Trajectory of planet')
    axes.set_xlabel('x [m]')
    axes.set_ylabel('y [m]')

    # Create Line2D objects to represent body and trail
    planet,  = axes.plot([],[],'o')
    trail,   = axes.plot([],[],'-')

    # Get arrays of coordinates to display (subset of provided data)
    frame_interval = 1/fps                  # time between frames [s]
    skip = round(frame_interval*speedup/dt) # number of calculated points per frame
    x_plot = x_arr[::skip]  # take every Nth value (N = skip)
    y_plot = y_arr[::skip]
    n_frames = len(x_plot)

    # Create and return animation object
    ani = animation.FuncAnimation(fig,animate_with_trail, frames=n_frames, interval=frame_interval*1000,
                                  fargs=[planet, trail, x_plot, y_plot])
    return ani

#### A4: Functional test
The following code calls all the functions above to create an animation. Given a finite computing power, `dt` is choosen to be 0.1 and the frames per second is choosen to be 60. Further analysis of optimising `dt` can be found below in section A5.

In [ ]:
dt_a = 0.1         #step size for animation [s]
fps= 60            #frames per second
speedup = 1000     #ratio of simulated time to screen time

plt.ioff()
fig, axes = plt.subplots()
axes.set_aspect('equal')

ani_dt = create_animation(x_test, y_test, dt_a, fps, speedup)

HTML(ani_dt.to_jshtml())

### A5: Effect of changing dt

On top of setting `dt` to be `YEAR/10000` (as used above), we can investigate how the animation changes when `dt` is set to `YEAR/1000`and `YEAR/10000`. All other variables are kept constant.

Note that there are two `dt` variables in creating this animation. The first is used as the time step (`dt_e`)and the second is used to decide how many frames are used in the animation (`dt_a`). I will changing the `dt` in Euler's equation because this actually affects the physics, making it more important. `dt_a` is kept constant at 0.01.

In [ ]:
#############
# LARGER DT #
#############
M_STAR  = 2.5e30     # mass of star [kg]
dt_e = YEAR/1000     # time step for Euler's method [s]
pos = (2.0e11, 0)    # initial position (x,y) [m]
vel = (0,2.89e4)     # initial velocity (vx,vy) [m/s]

# Calculate trajectory
x_dtlar, y_dtlar = trajectory(pos,vel,M_STAR,dt_e,2*YEAR)

dt_a = 0.1           # step size [s]
fps= 60              # frames per second
speedup = 1000       # ratio of simulated time to screen time

plt.ioff()
fig, axes = plt.subplots()
axes.set_aspect('equal')

ani_dtlar = create_animation(x_dtlar, y_dtlar, dt_a, fps, speedup)

HTML(ani_dtlar.to_jshtml())

In [ ]:
##############
# SMALLER DT #
##############
M_STAR  = 2.5e30     # mass of star [kg]
dt_e = YEAR/100000   # time step for Euler's method [s]
pos = (2.0e11, 0)    # initial position (x,y) [m]
vel = (0,2.89e4)     # initial velocity (vx,vy) [m/s]

# Calculate trajectory
x_dtsmall, y_dtsmall = trajectory(pos,vel,M_STAR,dt_e,2*YEAR)

dt_a = 0.1           # step size [s]
fps= 60              # frames per second
speedup = 1000       # ratio of simulated time to screen time

plt.ioff()
fig, axes = plt.subplots()
axes.set_aspect('equal')

ani_dtsmall = create_animation(x_dtsmall, y_dtsmall, dt_a, fps, speedup)

HTML(ani_dtsmall.to_jshtml())

#### A5: Analysis
`dt` is ideal when it is `YEAR/10000`. When it is larger (`YEAR/1000`), there are not enough points on the animation so it does not produce a smooth animation. When it is smaller (`YEAR/10000`), the animation is very smooth but it takes up too much memory and takes too long to compile.

### A6: Effect of stellar mass
I will now investigate the behaviour of the planet when the mass of the star is different. I will set stellar mass to be $3 \times 10^{30} kg$ and $2 \times 10^{30} kg$. `dt_e` is set to be `YEAR/10000` based on the investigation above and all other variables are kept constant.

In [ ]:
###############
# LARGER STAR #
###############
M_STAR  = 3e30       # mass of star [kg]
dt_e = YEAR/10000    # time step for Euler's method [s]
pos = (2.0e11, 0)    # initial position (x,y) [m]
vel = (0,2.89e4)     # initial velocity (vx,vy) [m/s]

# Calculate trajectory
x_bigstar, y_bigstar = trajectory(pos,vel,M_STAR,dt_e,2*YEAR)

dt_a = 0.1           # step size [s]
fps = 60             # frames per second
speedup = 1000       # ratio of simulated time to screen time

plt.ioff()
fig, axes = plt.subplots()
axes.set_aspect('equal')

ani_bigstar = create_animation(x_bigstar, y_bigstar, dt_a, fps, speedup)

HTML(ani_bigstar.to_jshtml())

In [ ]:
###############
# SMALLER SUN #
###############
M_STAR  = 2e30       # mass of star [kg]
dt_e = YEAR/10000    # time step for Euler's method [s]
pos = (2.0e11, 0)    # initial position (x,y) [m]
vel = (0,2.89e4)     # initial velocity (vx,vy) [m/s]

# Calculate trajectory
# duration of animation increased to show a full orbit
x_smallstar, y_smallstar = trajectory(pos,vel,M_STAR,dt_e,3*YEAR)

dt_a = 0.1           # step size [s]
fps = 60             # frames per second
speedup = 1000       # ratio of simulated time to screen time

plt.ioff()
fig, axes = plt.subplots()
axes.set_aspect('equal')

ani_smallstar = create_animation(x_smallstar, y_smallstar, dt_a, fps, speedup)

HTML(ani_smallstar.to_jshtml())

#### A6: Analysis
When the stellar mass is smaller, there is a weaker gravitaitonal force. This results in a smaller acceleration of the planet and thus, a small angular velocity. This is illustrated in the animation since when the stellar mass is smaller, the planet orbits the star slower. In addition, a smaller stellar mass shifts the orbit in the opposite direction of its starting position. The opposite can be said for cases when stellar mass is increased.

### A7: Effect of planet's starting position
I will investigate when the planet starts closer to the star and when it starts further away. Once again, all other variables are kept constant.

In [ ]:
#################
# CLOSER PLANET #
#################
M_STAR  = 2.5e30      # mass of star [kg]
dt_e = YEAR/10000     # time step for Euler's method [s]
pos = (1e11, 0)       # initial position (x,y) [m]
vel = (0,2.89e4)      # initial velocity (vx,vy) [m/s]

# Calculate trajectory
x_closeplanet, y_closeplanet = trajectory(pos,vel,M_STAR,dt_e,2*YEAR)

dt_a = 0.1            # step size [s]
fps = 60              # frames per second
speedup = 1000        # ratio of simulated time to screen time

plt.ioff()
fig, axes = plt.subplots()
axes.set_aspect('equal')

ani_closeplanet = create_animation(x_closeplanet, y_closeplanet, dt_a, fps, speedup)

HTML(ani_closeplanet.to_jshtml())

In [ ]:
##################
# FURTHER PLANET #
##################
M_STAR  = 2.5e30     # mass of star [kg]
dt_e = YEAR/10000   # time step for Euler's method [s]
pos = (3e11, 0)    # initial position (x,y) [m]
vel = (0,2.89e4)     # initial velocity (vx,vy) [m/s]

# Calculate trajectory
x_farplanet, y_farplanet = trajectory(pos,vel,M_STAR,dt_e,2*YEAR)

dt_a = 0.1          # step size [s]
fps= 60              # frames per second
speedup = 1000      # ratio of simulated time to screen time

plt.ioff()
fig, axes = plt.subplots()
axes.set_aspect('equal')

ani_farplanet = create_animation(x_farplanet, y_farplanet, dt_a, fps, speedup)

HTML(ani_farplanet.to_jshtml())

#### A7: Analysis
As the planet's starting position moves further away, it is less likely to be captured by the star's gravitational field so it ends up being slingshotted away. As the planet's starting position moves closer, it creates a more elliptical orbit which eventually also slingshots the planet away from the star.

### A8: Effect of planet's initial velocity
I will change the y-component in the planet's initial velocity and observe its effect. Once again, all other variables are kept constant.

In [ ]:
#################
# FASTER PLANET #
#################
M_STAR  = 2.5e30     # mass of star [kg]
dt_e = YEAR/10000    # time step for Euler's method [s]
pos = (2.0e11, 0)    # initial position (x,y) [m]
vel = (0,2.99e4)     # initial velocity (vx,vy) [m/s]

# Calculate trajectory
x_fasterplanet, y_fasterplanet = trajectory(pos,vel,M_STAR,dt_e,2*YEAR)

dt_a = 0.1           # step size [s]
fps = 60             # frames per second
speedup = 1000       # ratio of simulated time to screen time

plt.ioff()
fig, axes = plt.subplots()
axes.set_aspect('equal')

ani_fasterplanet = create_animation(x_fasterplanet, y_fasterplanet, dt_a, fps, speedup)

HTML(ani_fasterplanet.to_jshtml())

In [ ]:
#################
# SLOWER PLANET #
#################
M_STAR  = 2.5e30     # mass of star [kg]
dt_e = YEAR/10000   # time step for Euler's method [s]
pos = (2.0e11, 0)    # initial position (x,y) [m]
vel = (0,2.79e4)     # initial velocity (vx,vy) [m/s]

# Calculate trajectory
x_slowerplanet, y_slowerplanet = trajectory(pos,vel,M_STAR,dt_e,2*YEAR)

dt_a = 0.1          # step size [s]
fps= 60              # frames per second
speedup = 1000      # ratio of simulated time to screen time
y_slowerplanet
plt.ioff()
fig, axes = plt.subplots()
axes.set_aspect('equal')

ani_slowerplanet = create_animation(x_slowerplanet, y_slowerplanet, dt_a, fps, speedup)

HTML(ani_slowerplanet.to_jshtml())

#### A8: Analysis
As initial velocity increases, the orbit's focal point shifts to the opposite side of the planet's starting position.

## Part B: Two planets orbitting a star

The next part of this investigation will involve the illustration of a 3-body system involving two planets and a star. The parameters of the planets and stars are as follow:

- Mass of star: $2.5 \times 10^{30} kg$
- Mass of planets: $6 \times 10^{24} kg$

- Initial position of first planet: $(2, 0) \times 10^{11} m$
- Initial velocity of first planet: $(0, 2.89) \times 10^{4} m s^{-1}$

- Initial position of second planet: $(1.56, 1.56) \times 10^{11} m$
- Initial velocity of second planet: $(−1.95, 1.95) \times 10^4 m s^{-1}$

I will look at the case when the two planets do not interact with each other which will then be followed by an investigation of when the two planets do interact.

In [ ]:
#universal variables used throughout part b and c
G       = 6.6743e-11 # gravitational constant [m^3 kg^-1 s^-2]
YEAR    = 3.154e7    # one year in seconds
dt = YEAR/10000      # time step for Euler's method [s]

#initialising paramters of planets and star
M_STAR  = 2.5e30     # mass of star [kg]
m_planet = 6e24      # mass of planets [kg]

#planet 1
pos1 = (2.0e11, 0)    # initial position (x,y) [m]
vel1 = (0,2.89e4)     # initial velocity (vx,vy) [m/s]

#planet 2
pos2 = (1.56e11, 1.56e11)    # initial position (x,y) [m]
vel2 = (-1.95e4,1.95e4)     # initial velocity (vx,vy) [m/s]

In [ ]:
# functions to allow animation of multiple objects
# code taken from [3]
def animate_multiple(i, planets, trails, trajectories):
    """
    Update display for motion of multiple bodies with trails.
    
    Arguments:
        i:       frame number (from 0 at time = 0)
        planets: list of Line2D objects containing coordinates of each body to move
        trails:  list of Line2D objects containing coordinates of trails
        trajectories: list of trajectories, each of the form [x_arr, y_arr]
                 where x_arr and y_arr are arrays of x and y coordinates at
                 each time step
    
    Note that all trajectories must have the same number of points.
    """
    for j in range(len(trajectories)):    # for each trajectory ...
        trajectory = trajectories[j]
        x_arr, y_arr = trajectory         # get arrays of x and y coordinats
        x, y = x_arr[i], y_arr[i]
        planets[j].set_data([x],[y])
        trails[j].set_data(x_arr[:i+1],y_arr[:i+1]) # Trail has all points up to this one
        
def get_bodies_and_trails(axes, n):
    """
    Get the required number of Line2D objects to represent the bodies
    being animated, and the trails left behind them.
    
    Arguments:
        axes: the axes to be used for plotting the bodies
        n:    number of bodies in simulation
    
    Returns: (bodies, trails) where
        bodies is a list of Line2D objects to be plotted as discs
        trails is a list of Line2D objects to be plotted as lines
    """
    bodies = []
    trails = []
    for i in range(n):              # repeat n times
        body,  = axes.plot([],[],'o')           # Line2D with disc markers
        col    = body.get_color()               # get colour of body ...
        trail, = axes.plot([],[],'-',color=col) # and set trail (line) to same colour
        bodies.append(body)         # add Line2D objects to lists
        trails.append(trail)
    return bodies, trails

def create_animation_planets(trajectories, dt, fps, speedup, n):
    """
    Create an animation of objects with given trajectories.
    
    Arguments:
        trajectories: list of trajectories, each of the form [x_arr, y_arr]
                 where x_arr and y_arr are arrays of x and y coordinates at
                 each time step
        dt:           time step in simulation [s]
        fps:          number of frames per second for animation
        speedup:      ratio of simulated time to screen time
        n:            number of bodies
    """
    # Find range of coordinates to include full trajectories 
    r_max = 1.05 * np.max(np.abs(trajectories))

    # Create and configure figure and axes
    plt.ioff()
    fig, axes = plt.subplots()
    axes.set_xlim(-r_max,r_max)
    axes.set_ylim(-r_max,r_max)
    axes.set_aspect('equal')

    # Create Line2D objects to represent bodies and trails
    bodies, trails = get_bodies_and_trails(axes, n)
    body = bodies[0]
    trail = trails[0]

    # Get arrays of coordinates to display (subset of provided data)
    frame_interval = 1/fps                  # time between frames [s]
    skip = round(frame_interval*speedup/dt) # number of calculated points per frame
    plot_trajectories = []
    for trajectory in trajectories:
        x_arr, y_arr = trajectory
        x_plot = x_arr[::skip]  # take every Nth value (N = skip)
        y_plot = y_arr[::skip]
        plot_trajectory = [x_plot, y_plot]
        plot_trajectories.append(plot_trajectory)
        n_frames = len(x_plot)
    
    # Create and return animation object
    ani = animation.FuncAnimation(fig,animate_multiple, frames=n_frames, interval=frame_interval*1000,
                                  fargs=[bodies, trails, plot_trajectories])
    return ani

### B1: Two non-interacting planets
The following is an animation of the two planets which only experience the gravitaitonal force of the star.

In [ ]:
# code adapted from [3]
# Calculate trajectories (lists of x and y coordinates) [m]
x1, y1 = trajectory(pos1,vel1,M_STAR,dt,2*YEAR)
x2, y2 = trajectory(pos2,vel2,M_STAR,dt,2*YEAR)

# Put the results into a list of trajectories, with each trajectory being
# a list containing the lists of x and y coordinates.
traj1 = [x1, y1]
traj2 = [x2, y2]
trajectories = [traj1, traj2]

# One year is represented by 2 seconds in the animation
speedup = YEAR/2

# Create and display the animation
ani_nonint = create_animation_planets(trajectories, dt, 50, speedup, 2)
HTML(ani_nonint.to_jshtml())

### B2: Two interacting planets
We now need to consider the force planet 1 exerts on planet 2 and vice versa. Therefore, the functions `move_planet` and `trajectory` need to be changed. 

In [ ]:
def move_planet_int2(p1, v1, m1, p2, v2, m2, m_star, dt):
    """
    Calculate motion of 2 planets who experience the gravitaitonal field of a neibhouring star
    and each other.
    
    Input:
      - p1: position vector of planet one at start of time step (NumPy array) [m]
      - v1: velocity vector of planet one at start of time step (NumPy array) [m/s]
      - m1: mass of planet one (float) [kg]
      - p2: position vector of planet two at start of time step (NumPy array) [m]
      - v2: velocity vector of planet two at start of time step (NumPy array)[m/s]
      - m2: mass of planet two (float) [kg]
      - m_star: mass of star (float) [kg]
      - dt: time step (float) [s]
      
    Output: (p1_new, v1_new, p2_new, v2_new)
      - p1_new: position vector of planet one at end of time step (NumPy array) [m]
      - v1_new: velocity vector of planet one at end of time step (NumPy array) [m/s]
      - p2_new: position vector of planet two at end of time step (NumPy array) [m]
      - v2_new: velocity vector of planet two at end of time step (NumPy array) [m/s]
    
    Depends on:
      - force: function to calculate the gravitational force between two objects based on their
      position and masses.
    """
    #set starting star posiiton
    star_pos = np.array([0,0])
    
    #stellar force on planet 1
    star_force1 = force(p1, star_pos, m1, m_star)
    #stellar force on planet 2
    star_force2 = force(p2, star_pos, m2, m_star)
    
    #force of planet 2 on planet 1
    plan_force1 = force(p1, p2, m1, m2)
    #force of planet 1 on planet 2
    plan_force2 = force(p2, p1, m2, m1)
    
    #force experienced by planet 1
    force1 = star_force1 + plan_force1
    #force experienced by planet 2
    force2 = star_force2 + plan_force2   
    
    #acceleration experienced by planet 1
    accel1 = force1/m1
    #acceleration experienced by planet 2
    accel2 = force2/m2
    
    #calculate new velocity+position of planet 1
    v1_new = np.array(v1) + np.array(accel1)*dt
    p1_new = np.array(p1) + np.array(v1)*dt

    #calculate new velocity+position of planet 2
    v2_new = v2 + accel2*dt
    p2_new = np.array(p2) + np.array(v2)*dt
    
    return p1_new, v1_new, p2_new, v2_new



def trajectory_int2(pos1, vel1, m1, pos2, vel2, m2, m_star, dt, t_max):
    """
    Calculate trajectory of two planet from given their starting position, using Euler's method.
    
    Input:
      - pos1: position vector of planet one at start of simulation [m] (NumPy array)
      - vel1: velocity vector of planet one at start of simulation [m/s] (NumPy array)
      - m1: mass of planet one (float) [kg]
      - pos2: position vector of planet two at start of simulation [m] (NumPy array)
      - vel2: velocity vector of planet two at start of simulation [m/s] (NumPy array)
      - m2: mass of planet two (float) [kg]
      - m_star:   mass of star [kg]
      - dt:       time step    [s]
      - t_max:    duration of calculated motion [s]

    Output: (x1_arr, y1_arr, x2_arr, y2_arr) [m]
        where all outputs are NumPy arrays containing the x and y coordinates of
        the planets at each time step, starting with the initial position
        
    Depends:
      - move_planet_int2: function that applies Euler's method to find the new position of two planets given
      that they exert forces each other and they experience the force of a star.
    """
    #array of positions
    x1_arr, y1_arr = [pos1[0]] , [pos1[1]]
    x2_arr, y2_arr = [pos2[0]] , [pos2[1]]
    #total runtime in s
    time = 0
    while time <= t_max:
        #call function to calculate new position and velocity
        pos1, vel1, pos2, vel2 = move_planet_int2(pos1, vel1, m1, pos2, vel2, m2, m_star, dt)
        x1,y1 = pos1
        x1_arr.append(x1)
        y1_arr.append(y1)
        x2,y2 = pos2
        x2_arr.append(x2)
        y2_arr.append(y2) 
        time += dt
    return x1_arr, y1_arr,x2_arr, y2_arr

In [ ]:
# code adapted from [3]
# calculate trajectories (lists of x and y coordinates) [m]
# masses of both planets are the same so both have the value of 'm_planet'
x1_int, y1_int, x2_int, y2_int= trajectory_int2(pos1, vel1, m_planet, pos2, vel2, m_planet, M_STAR, dt, 2*YEAR)

# Put the results into a list of trajectories, with each trajectory being
# a list containing the lists of x and y coordinates.
traj1_int = [x1_int, y1_int]
traj2_int = [x2_int, y2_int]
trajectories_int = [traj1_int, traj2_int]

# One year is represented by 2 seconds in the animation
speedup = YEAR/2

# Create and display the animation
ani_int = create_animation_planets(trajectories_int, dt, 50, speedup, 2)
HTML(ani_int.to_jshtml())

In [ ]:
#print final positions of both planets when not interacting and interacting
print("NOT considering gravitational force between planets")
print("final position of blue dot1 (x,y):")
print(x1[len(x1)-1],y1[len(y1)-1])
print(f"final position of orange dot1 (x,y):")
print(x2[len(x2)-1],y2[len(y2)-1])
print("")
print("CONSIDERING gravitational force between planets")
print("final position of blue dot1 (x,y):")
print(x1_int[len(x1_int)-1],y1_int[len(y1_int)-1])
print(f"final position of orange dot1 (x,y):")
print(x2_int[len(x2_int)-1],y2_int[len(y2_int)-1])

#### B1 and B2: Analysis
It is difficult to tell the difference between the two animations. However, by looking at the final coordinates shown in the code above, we can see that there is indeed a difference.

### B3: One planet significantly larger than the other
To make the planet-to-planet interaction more obvious, this part will increase the mass of the second planet by a factor of 100.

In [ ]:
# code adapted from [3]
m1 = 6e24
m2 = 6e26
# Calculate trajectories (lists of x and y coordinates) [m]
x1_obv, y1_obv, x2_obv, y2_obv= trajectory_int2(pos1,vel1,m1,pos2,vel2,m2,M_STAR,dt,2*YEAR)

# Put the results into a list of trajectories, with each trajectory being
# a list containing the lists of x and y coordinates.
traj1_obv = [x1_obv, y1_obv]
traj2_obv = [x2_obv, y2_obv]
trajectories_obv = [traj1_obv, traj2_obv]

# One year is represented by 2 seconds in the animation
speedup = YEAR/2

# Create and display the animation
aniobv = create_animation_planets(trajectories_obv, dt, 50, speedup, 2)
HTML(aniobv.to_jshtml())

#### B4: Analysis
The second planet with higher mass (shown in orange) maintains its circular orbit. However, the smaller planet (shown in blue) experiences too much force so it does not remain in a stable circular orbit. It flies off to infinity.

## Part C: Further investigation
In this section I will be extending on the work above. My first investigation will be the modelling of a system consisting of a star, planet and a moon. To do this, I will use data collected on the Earth and Earth's moon to decide the parameters.

My second investigation will be.....

### C1: System of star, planet, and moon
In order to decide the velocity parameters, I will be utilising the following equation [4]:
$$V_{orb} = \sqrt{\frac{GM}{R}} (5)$$

where G is $6.6743 \times 10^{-11} N m^2 kg^{-2}$ [2]. 

The orbital radius and mass of Earth and the Sun are used as parameters. These parameters are then used to calculate the velocities.

#### Star parameters
Star's mass: $1.989 \times 10^{30} kg$
- The Sun's mass is $1.989 \times 10^{30} kg$ [5]

##### Planet parameters
Planet's position: $(1.47152555,0) \times 10^{11} m$ 
- The distance of the Earth from the sun is about 147,152,555 km [6]

Planet's mass: $5.97217e \times 10^{24} kg$ 
- Earth's mass is $5.97217 \times 10^{24} kg$ [7]

Planet's velocity: $(0, 30035.60262) m s^{-1}$
- Using equation (5), the Sun's mass, and Earth's orbital radius, the orbital velocity is found to be $30035.60262 m s^{-1}$.

##### Moon parameters
Moon's position: $(1.47536955,0) \times 10^{11} m$
- The distance from the Earth to the sun is about 354,400 km [8]. This value is added to the planet's position.

Moon's mass: $7.346 \times 10^{22} kg$
- The moon's mass is $7.346 \times 10^{22} kg$  [9].

Moon's velocity: $(0, 3.109613049)\times 10^{4} m s^{-1}$
- Using equation (5), Earth' mass, and our moon's orbital radius, the orbital velocity is found to be $1060.52787 m s^{-1}$. Adding this to Earth's velocity, we get a vector velocity of $(0, 31096.13049) m s^{-1}$. This is the closest model I could derive given the precision of my sources.

In [ ]:
#initialising paramters of planets and star
M_STAR  = 1.989e30     # mass of star [kg]

#planet
pos_planet = (1.47152555e11, 0)     # initial position (x,y) [m]
vel_planet = (0, 30035.60262)           # initial velocity (vx,vy) [m/s]
m_planet = 5.97217e24              # mass of planet [kg]

#moon
pos_moon = (1.47536955e11, 0)      # initial position (x,y) [m]          
vel_moon = (0, 31096.13049) # initial velocity (vx,vy) [m/s]
m_moon = 0.07346e24             # mass of moon [kg]

In [ ]:
# code adapted from [3]
dt = YEAR/10000

# Calculate trajectories (lists of x and y coordinates) [m]
x_planet, y_planet, x_moon, y_moon = trajectory_int2(pos_planet,vel_planet,m_planet,pos_moon,vel_moon,m_moon,M_STAR,dt,YEAR)

# Put the results into a list of trajectories, with each trajectory being
# a list containing the lists of x and y coordinates.
traj_planet = [x_planet, y_planet]
traj_moon = [x_moon, y_moon]
trajectories_moon = [traj_planet, traj_moon]

# One year is represented by 2 seconds in the animation
speedup = YEAR/2

# Create and display the animation
animoon_int = create_animation_planets(trajectories_moon, dt, 50, speedup, 2)
HTML(animoon_int.to_jshtml())

#### C1: Analysis
As we can see, the moon and the planet orbit the star together. The animation gives the impression that they travel along the same line, however, there is some slight deviation throughout the cycle.

### C2: Three-body problem
The three-body problem [10] is the almost completely random motion of three celestial bodies. As the number of celestial bodies in a system increase, the number of unknown variables increases, increasing the entropy of the system. Any slight change to the initial parameters would drastically change how the system behaves. In this section, I will be adding another planet to the system (making a system of 3 planets and a star). I will then alter the starting position of one of the objects from $(0,3\times 10^{11}) m$ to $(0,5 \times 10^{11}) m$.

In [ ]:
# code taken from [3] to accommodate 3 objects
def move_planet_int3(p1, v1, m1, p2, v2, m2, p3, v3, m3, m_star, dt):
    """
    Calculate motion of 3 planets who experience the gravitaitonal field of a neibhouring star
    and each other.
    
    Input:
      - p1: position vector of planet one at start of time step (NumPy array) [m]
      - v1: velocity vector of planet one at start of time step (NumPy array) [m/s]
      - m1: mass of planet one (float) [kg]
      - p2: position vector of planet two at start of time step (NumPy array) [m]
      - v2: velocity vector of planet two at start of time step (NumPy array)[m/s]
      - m2: mass of planet two (float) [kg]
      - p3: position vector of planet three at start of time step (NumPy array) [m]
      - v3: velocity vector of planet three at start of time step (NumPy array)[m/s]
      - m3: mass of planet three (float) [kg]
      - m_star: mass of star (float) [kg]
      - dt: time step (float) [s]
      
    Output: (p1_new, v1_new, p2_new, v2_new, p3_new, v3_new)
      - p1_new: position vector of planet one at end of time step (NumPy array) [m]
      - v1_new: velocity vector of planet one at end of time step (NumPy array) [m/s]
      - p2_new: position vector of planet two at end of time step (NumPy array) [m]
      - v2_new: velocity vector of planet two at end of time step (NumPy array) [m/s]
      - p3_new: position vector of planet three at end of time step (NumPy array) [m]
      - v3_new: velocity vector of planet three at end of time step (NumPy array) [m/s]
    
    Depends on:
      - force: function to calculate the gravitational force between two objects based on their
      position and masses.
    """
    #set starting star posiiton
    star_pos = np.array([0,0])
    
    #stellar force on planet 1
    star_force1 = force(p1, star_pos, m1, m_star)
    #stellar force on planet 2
    star_force2 = force(p2, star_pos, m2, m_star)
    #stellar force on planet 3
    star_force3 = force(p3, star_pos, m3, m_star)
    
    #force of planet 2 on planet 1
    plan_force21 = force(p1, p2, m1, m2)
    #force of planet 3 on planet 1
    plan_force31 = force(p1, p3, m1, m3)
    
    #force of planet 1 on planet 2
    plan_force12 = force(p2, p1, m2, m1)
    #force of planet 3 on planet 2
    plan_force32 = force(p2, p3, m2, m3)
    
    #force of planet 1 on planet 3
    plan_force13 = force(p3, p1, m3, m1)
    #force of planet 2 on planet 3
    plan_force23 = force(p3, p2, m3, m2)  
    
    #force experienced by planet 1
    force1 = star_force1 + plan_force21 + plan_force31 
    #force experienced by planet 2
    force2 = star_force2 + plan_force12 + plan_force32
    #force experienced by planet 3
    force3 = star_force3 + plan_force13 + plan_force23
    
    #acceleration experienced by planet 1
    accel1 = force1/m1
    #acceleration experienced by planet 2
    accel2 = force2/m2
    #acceleration experienced by planet 3
    accel3 = force3/m3
    
    #calculate new velocity+position of planet 1
    v1_new = np.array(v1) + np.array(accel1)*dt
    p1_new = np.array(p1) + np.array(v1)*dt

    #calculate new velocity+position of planet 2
    v2_new = v2 + accel2*dt
    p2_new = np.array(p2) + np.array(v2)*dt

    #calculate new velocity+position of planet 3
    v3_new = v3 + accel3*dt
    p3_new = np.array(p3) + np.array(v3)*dt
    
    return p1_new, v1_new, p2_new, v2_new, p3_new, v3_new


def trajectory_int3(pos1, vel1, m1, pos2, vel2, m2, pos3, vel3, m3, m_star, dt, t_max):
    """
    Calculate trajectory of two planet from given their starting position, using Euler's method.
    
    Input:
      - pos1: position vector of planet one at start of simulation [m] (NumPy array)
      - vel1: velocity vector of planet one at start of simulation [m/s] (NumPy array)
      - m1: mass of planet one (float) [kg]
      - pos2: position vector of planet two at start of simulation [m] (NumPy array)
      - vel2: velocity vector of planet two at start of simulation [m/s] (NumPy array)
      - m2: mass of planet two (float) [kg]
      - pos3: position vector of planet three at start of simulation [m] (NumPy array)
      - vel3: velocity vector of planet three at start of simulation [m/s] (NumPy array)
      - m3: mass of planet three (float) [kg]
      - m_star:   mass of star [kg]
      - dt:       time step    [s]
      - t_max:    duration of calculated motion [s]

    Output: (x1_arr, y1_arr, x2_arr, y2_arr, x3_arr, y3_arr) [m]
        where all outputs are NumPy arrays containing the x and y coordinates of
        the planets at each time step, starting with the initial position
        
    Depends:
      - move_planet_int3: function that applies Euler's method to find the new position of two planets given
      that they exert forces each other and they experience the force of a star.
    """
    #array of positions
    x1_arr, y1_arr = [pos1[0]] , [pos1[1]]
    x2_arr, y2_arr = [pos2[0]] , [pos2[1]]
    x3_arr, y3_arr = [pos3[0]] , [pos3[1]]
    #total runtime in s
    time = 0
    while time <= t_max:
        #call function to calculate new position and velocity
        pos1, vel1, pos2, vel2, pos3, vel3 = move_planet_int3(pos1, vel1, m1, pos2, vel2, m2, pos3, vel3, m3, m_star, dt)
        x1,y1 = pos1
        x1_arr.append(x1)
        y1_arr.append(y1)
        x2,y2 = pos2
        x2_arr.append(x2)
        y2_arr.append(y2)
        x3,y3 = pos3
        x3_arr.append(x3)
        y3_arr.append(y3) 
        time += dt
    return x1_arr, y1_arr, x2_arr, y2_arr, x3_arr, y3_arr

In [ ]:
###################
# PLANET 3 CLOSER #
###################
#initialising paramters of planets and star
M_STAR  = 2.5e30      # mass of star [kg]

#planet 1
pos1 = (4.0e11, 0)    # initial position (x,y) [m]
vel1 = (0,4e4)        # initial velocity (vx,vy) [m/s]
m1 = 5e24             # mass of planets [kg]

#planet 2
pos2 = (2e11, 2e11)   # initial position (x,y) [m]
vel2 = (-2e4,2e4)     # initial velocity (vx,vy) [m/s]
m2 = 5e26             # mass of planets [kg]

#planet 3
pos3 = (0,3e11)       # initial position (x,y) [m]
vel3 = (2e4,0)        # initial velocity (vx,vy) [m/s]
m3 = 5e25             # mass of planets [kg]

# code adapted from [3]
# calculate trajectories (lists of x and y coordinates) [m]
# masses of both planets are the same so both have the value of 'm_planet'
x1_int3, y1_int3, x2_int3, y2_int3, x3_int3, y3_int3= trajectory_int3(pos1, vel1, m1, pos2, vel2, m2, pos3, vel3, m3, M_STAR, dt, 2*YEAR)

# Put the results into a list of trajectories, with each trajectory being
# a list containing the lists of x and y coordinates.
traj1_int3 = [x1_int3, y1_int3]
traj2_int3 = [x2_int3, y2_int3]
traj3_int3 = [x3_int3, y3_int3]
trajectories_int3 = [traj1_int3, traj2_int3, traj3_int3]

# One year is represented by 2 seconds in the animation
speedup = YEAR/2

# Create and display the animation
ani_int3 = create_animation_planets(trajectories_int3, dt, 50, speedup, 3)
HTML(ani_int3.to_jshtml())

In [ ]:
#########################
# PLANET 3 FURTHER AWAY #
#########################
#initialising paramters of planets and star
M_STAR  = 2.5e30      # mass of star [kg]

#planet 1
pos1 = (4.0e11, 0)    # initial position (x,y) [m]
vel1 = (0,4e4)        # initial velocity (vx,vy) [m/s]
m1 = 5e24             # mass of planets [kg]

#planet 2
pos2 = (2e11, 2e11)   # initial position (x,y) [m]
vel2 = (-2e4,2e4)     # initial velocity (vx,vy) [m/s]
m2 = 5e26             # mass of planets [kg]

#planet 3
pos3 = (0,5e11)       # initial position (x,y) [m]
vel3 = (2e4,0)        # initial velocity (vx,vy) [m/s]
m3 = 5e25             # mass of planets [kg]

# code adapted from [3]
# calculate trajectories (lists of x and y coordinates) [m]
# masses of both planets are the same so both have the value of 'm_planet'
x1_int3, y1_int3, x2_int3, y2_int3, x3_int3, y3_int3= trajectory_int3(pos1, vel1, m1, pos2, vel2, m2, pos3, vel3, m3, M_STAR, dt, 2*YEAR)

# Put the results into a list of trajectories, with each trajectory being
# a list containing the lists of x and y coordinates.
traj1_int3 = [x1_int3, y1_int3]
traj2_int3 = [x2_int3, y2_int3]
traj3_int3 = [x3_int3, y3_int3]
trajectories_int3 = [traj1_int3, traj2_int3, traj3_int3]

# One year is represented by 2 seconds in the animation
speedup = YEAR/2

# Create and display the animation
ani_int3 = create_animation_planets(trajectories_int3, dt, 50, speedup, 3)
HTML(ani_int3.to_jshtml())

#### C2: Analysis
As we can see from the two animations above, changing the initial parameters will drastically change the motion of the objects. Although they seem to ressemble each other, we must look at the range of the axis. Even a slight visible change in the animation would suggest a massive change if observed in person.

## Conclusion
In this notebook we have illustrated different types of planetary orbits. We have concluded that changing any parameters, no matter the number of celestial bodies, will result in some change of the outcome. As an extension, one could further increase the number of celestial bodies or try to add a z-component to the animation to deal in 3-dimensions.

## References
[1] Waugh, B. Chislett, B. "Unit 9: Simulating a physical system". PHAS0007: Computing. Accessed 6th of December, 2022.

[2] Waugh, B. Chislett, B. "Template notebook for final assignment". PHAS0007: Computing. Accessed 19th of December, 2022.

[3] Waugh, B. "Simulating orbits of two planets". PHAS0007: Computing. Accessed 19th of December, 2022.

[4] Lissauer, J. J. de Pater, I. "Fundamental Planetary Sciences : physics, chemistry, and habitability". Cambridge University Press. 2019. p. 604.

[5]"SOLAR SYSTEM EXPLORATION: Our Galactic Neighborhood. Sun". NASA. Accessed 23rd of December, 2022. https://solarsystem.nasa.gov/solar-system/sun/overview/

[6] "SOLAR SYSTEM EXPLORATION: Our Galactic Neighborhood. Earth". NASA. Accessed 19th of December, 2022. https://solarsystem.nasa.gov/planets/earth/overview/

[7] Williams, D. R. "Earth Fact Sheet". NASA. Accessed 23rd of December, 2022. https://nssdc.gsfc.nasa.gov/planetary/factsheet/earthfact.html

[8]"SOLAR SYSTEM EXPLORATION: Our Galactic Neighborhood. Moon". NASA. Accessed 19th of December, 2022. https://solarsystem.nasa.gov/moons/earths-moon/overview/

[9] Williams, D. R. "Moon Fact Sheet". NASA. Accessed 23rd of December, 2022. https://nssdc.gsfc.nasa.gov/planetary/factsheet/moonfact.html

[10] "three-body problem". Britannica: The Editors of Encyclopaedia Britannica. Accessed 23rd of December, 2022. https://www.britannica.com/science/three-body-problem